In [1]:
import pandas as pd
import numpy as np 
import sys
sys.path.append("../tax-calculator")
from taxcalc.records import Records
from taxcalc import Policy, Records, Calculator, Behavior, behavior
from taxcalc.utils import *
from numpy.testing import assert_array_almost_equal
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib.transforms import BlendedGenericTransform
%matplotlib inline
import copy

In [2]:
puf = pd.read_csv("../tax-calculator/puf.csv")
policy_base = Policy()
records_base = Records(puf)

policy_reform = Policy()
records_reform = Records(puf)

calcbase = Calculator(policy = policy_base, records = records_base)
calcreform = Calculator(policy = policy_reform, records = records_reform)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [3]:
reform = {
          2015:{
        "_ID_InterestPaid_HC":[0.5],
        "_ID_StateLocalTax_HC":[0.5],
        "_ID_Charity_HC":[0.5],
        "_ID_RealEstate_HC":[0.4]
    }
}
policy_reform.implement_reform(reform)

In [4]:
calcbase.advance_to_year(2015)
calcreform.advance_to_year(2015)

In [5]:
calcbase.calc_all()
calcreform.calc_all()

In [6]:
def agg_diff(calcX, calcY):
    df_x = results(calcX)
    df_y = results(calcY)
    
    agg_x = sum(df_x['_combined']*df_x['s006'])
    agg_y = sum(df_y['_combined']*df_y['s006'])
    
    agg_diif = agg_y - agg_x

    agg_diif.tofile('agg_diif.csv',sep=',',format='%1.3f')

In [7]:
agg_diff(calcbase,calcreform)

In [8]:
def agg_num_delta(calcX, calcY):
    df_x = results(calcX)
    df_y = results(calcY)
    
    id_itemizers_x = ((calcX.records.c04470 > 0) & (calcX.records.c00100 > 0))
    id_itemizers_y = ((calcY.records.c04470 > 0) & (calcY.records.c00100 > 0))
    df_x['pct_itm'] = id_itemizers_x
    df_y['pct_itm'] = id_itemizers_y
    
    itm_x = sum(df_x['pct_itm'] * df_x['s006'])
    itm_y = sum(df_y['pct_itm'] * df_y['s006'])
    
    agg_num_delta = itm_y - itm_x

    agg_num_delta.tofile('agg_num_delta.csv',sep=',',format='%1.3f')

In [9]:
agg_num_delta(calcbase,calcreform)

In [10]:
def func(reform_on_II, orgi_reform):
    policy_func = Policy()
    records_func = Records(puf)
    calc_func = Calculator(policy = policy_func, records = records_func)
    policy_bench = Policy()
    records_bench = Records(puf)
    calc_bench = Calculator(policy = policy_bench, records = records_bench)
    reform = {
        2015:{
        "_II_rt1":[max(0.1 - reform_on_II,0.0)],
        "_II_rt2":[max(0.15 - reform_on_II,0.0)],
        "_II_rt3":[max(0.25 - reform_on_II,0.0)],
        "_II_rt4":[max(0.28 - reform_on_II,0.0)],
        "_II_rt5":[max(0.33 - reform_on_II,0.0)],
        "_II_rt6":[max(0.35 - reform_on_II,0.0)],
        "_II_rt7":[max(0.396 - reform_on_II,0.0)]}
    }
    policy_func.implement_reform(reform)
    policy_func.implement_reform(orgi_reform)
    calc_func.advance_to_year(2015)
    calc_bench.advance_to_year(2015)
    calc_func.calc_all()
    calc_bench.calc_all()
    return ((calc_bench.records._combined*calc_bench.records.s006).sum()-(calc_func.records._combined*calc_func.records.s006).sum())


In [11]:
def reform_equiv(orgi_reform, epsilon):
    upp = 0.1
    low = 0
    mid = (upp + low)/2.0
    while (upp - low)/2.0 > epsilon:
        delta = func(mid,orgi_reform)
        if delta == 0:
            return mid 
        elif delta < 0:
            low = mid
        else:
            upp = mid
        mid = (upp + low)/2.0
    return mid

In [12]:
reform_equiv(reform, 0.0001)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been ex

0.01767578125

In [13]:
func(0.01767578125,reform) / (calcbase.records._combined*calcbase.records.s006).sum()

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


-1.9704763792789642e-05

In [14]:
reform1 = {
          2015:{
                '_ID_StateLocalTax_HC': [0],
                '_ID_RealEstate_HC': [0],
                '_ID_InterestPaid_HC': [0],
                '_ID_Charity_HC': [0]}
}
reform2 = {
          2015:{
                '_ID_StateLocalTax_HC': [1],
                '_ID_RealEstate_HC': [1],
                '_ID_InterestPaid_HC': [0],
                '_ID_Charity_HC': [0]}
}
reform3 = {
          2015:{
                '_ID_StateLocalTax_HC': [1],
                '_ID_RealEstate_HC': [1],
                '_ID_InterestPaid_HC': [0],
                '_ID_Charity_HC': [1]}
}
reform4 = {
          2015:{
                '_ID_StateLocalTax_HC': [1],
                '_ID_RealEstate_HC': [1],
                '_ID_InterestPaid_HC': [1],
                '_ID_Charity_HC': [0]}
}
reform5 = {
          2015:{
                '_ID_StateLocalTax_HC': [0],
                '_ID_RealEstate_HC': [0],
                '_ID_InterestPaid_HC': [1],
                '_ID_Charity_HC': [1]}
}
reform6 = {
          2015:{
                '_ID_StateLocalTax_HC': [0],
                '_ID_RealEstate_HC': [0],
                '_ID_InterestPaid_HC': [0],
                '_ID_Charity_HC': [1]}
}
reform7 = {
          2015:{
                '_ID_StateLocalTax_HC': [0],
                '_ID_RealEstate_HC': [0],
                '_ID_InterestPaid_HC': [1],
                '_ID_Charity_HC': [0]}
}
reform8 = {
          2015:{
                '_ID_StateLocalTax_HC': [1],
                '_ID_RealEstate_HC': [1],
                '_ID_InterestPaid_HC': [1],
                '_ID_Charity_HC': [1]}
}


In [15]:
reform_equiv(reform1, 0.0001)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been ex

9.765625e-05

In [16]:
reform_equiv(reform2, 0.0001)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been ex

0.01533203125

In [17]:
reform_equiv(reform3, 0.0001)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been ex

0.020214843750000003

In [18]:
reform_equiv(reform4, 0.0001)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been ex

0.022167968750000003

In [19]:
reform_equiv(reform5, 0.0001)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been ex

0.019824218750000004

In [20]:
reform_equiv(reform6, 0.0001)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been ex

0.00791015625

In [21]:
reform_equiv(reform7, 0.0001)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been ex

0.01376953125

In [22]:
reform_equiv(reform8, 0.0001)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been ex

0.02529296875